<a href="https://colab.research.google.com/github/Svensone/Web-Scraping/blob/master/WebScraping_Covid_Bali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary

1) get covid cases per regency in Bali

2) additional data for Bali Province
- population and area of regencies
- hospital beds

3) comparison on national level


In [ ]:
## install necessary dependencies etc.

!pip install scrapy

In [70]:
## import dependencies

# Web Scraping
import scrapy
import requests
from bs4 import BeautifulSoup
import time
import csv

# Data processing
import pandas as pd


## 1. Covid Cases for Bali on Regencies level

### scrapying data

In [5]:
url = "https://infocorona.baliprov.go.id/API/pendataan/laporan-harian-01.php"
headers = {'User-Agent': 'Mozilla/5.0'}

In [6]:
response = requests.get(url)

In [7]:
response.status_code

200

In [8]:
response.content

b'<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" >\n<link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.7.2/css/all.css">\n<link rel="stylesheet" href="https://pendataan.baliprov.go.id/assets/css/style.css?v=15">\n<link rel="stylesheet" href="https://pendataan.baliprov.go.id/assets/css/widget-custom.css?v=1">\n\n\n<section class="section">\n<div class="section-header" style="margin-top: 18px;"><h1>Laporan Harian Berdasarkan Sumber Penularan pada Tiap Wilayah</h1></div>\n<div class="section-body"><div class="card"><div class="card-body">\n\n\n<div class=\'alert alert-info\' role=\'alert\'>\n    <b>Informasi!</b> Menampilkan data laporan tanggal 19 Nov 2020 . Karena data tanggal 20 Nov 2020 belum dipublish.\n</div>\n<div class="table-responsive text-center">\n    <table class="table table-bordered table-striped text-center" border="1">\n    <thead>\n        <tr>\n            <th rowspan="3">NO</th>\n            <th 

In [9]:
soup = BeautifulSoup(response.content, 'html.parser')

## extract relevant data

In [11]:
covid_tables = soup.find_all('table')

In [12]:
len(covid_tables)

1

In [32]:
## get update-date

## MISSING 
## write code to check if updated

## if not yet updated div shows:
## \nInformasi! Menampilkan data laporan tanggal 19 Nov 2020 . Karena data tanggal 20 Nov 2020 belum dipublish.\n

## if updated div shows:
## 

dateUpdated = soup.find('div', class_='alert alert-info').get_text()

# print(str(dateUpdated[-11:-2]))
if dateUpdated[-11:-2]=='dipublish':
  print('No new update yet available')
else:
  date = dateUpdated[-13:-2]

No new update yet available


In [33]:
## extract rows of table

table = covid_tables[0]
rows = table.find_all('tr')


In [ ]:
## write data to csv file

csvFile = open('CovidBali.csv', 'a')
writer = csv.writer(csvFile)

try:
  # write update-date
  writer.writerow(['Data from', date, 'from https://infocorona.baliprov.go.id/'])

  # get cell data
  for row in rows:
    csvRow = []
    for cell in row.find_all(['td', 'th']):
      csvRow.append(cell.get_text())
    writer.writerow(csvRow)
finally:
  csvFile.close()

In [ ]:
## check csv File

with open('CovidBali.csv', 'r') as f:
  for row in f:
    print(row)

Data from,19 Nov 2020,from https://infocorona.baliprov.go.id/

NO,KABUPATEN / KOTA,SUMBER PENULARAN POSITIF COVID-19 DI BALI

PPLN / PMI,PPDN,Transmisi Lokal,Lainnya,TOTAL

+,JMLH,+,JMLH,+,JMLH,+,JMLH,+,JMLH

1,JEMBRANA,0,26,0,11,5,459,0,0,5,496

2,TABANAN,0,21,0,8,15,931,0,0,15,960

3,BADUNG,0,27,0,4,15,2229,0,0,15,2260

4,DENPASAR,0,48,0,21,28,3527,0,0,28,3596

5,GIANYAR,0,29,0,12,14,1685,0,0,14,1726

6,BANGLI,0,58,0,3,0,808,0,0,0,869

7,KLUNGKUNG,0,19,0,0,0,891,0,0,0,910

8,KARANGASEM,0,23,0,0,6,944,0,0,6,967

9,BULELENG,0,41,0,6,5,1097,0,0,5,1144

10,KABUPATEN LAINNYA,0,5,0,30,0,11,0,0,0,46

11,WARGA NEGARA ASING,0,8,0,2,0,21,0,0,0,31

TOTAL,0,305,0,97,88,12603,0,0,88,13005



In [ ]:
## 2. additional Data

### Population and Area


In [83]:
url = 'https://id.wikipedia.org/wiki/Daftar_kabupaten_dan_kota_di_Bali#cite_note-:0bali-1'
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url)
print(response.status_code)

soup = BeautifulSoup(response.content, "html.parser")

#find table and extract data
table2 = soup.find_all('table', class_="wikitable sortable mw-collapsible")

## create df
## get 'th' data for columns name

allRows = soup.find_all('tr')
headerRow = allRows[0].find_all('th')


header = []
for cell in headerRow:
  data = cell.get_text()
  data = data.rstrip()
  data = data.replace('/', ' ')
  ## clean data from '[1]' Source link
  # regex1 = /\[.\]/g
  # data = data.replace(regex1, '')
  header.append(data)

header

200


[<tr>
 <th>No.
 </th>
 <th>Kabupaten/kota
 </th>
 <th>Pusat pemerintahan
 </th>
 <th>Bupati/wali kota
 </th>
 <th>Luas wilayah (km<sup>2</sup>)<sup class="reference" id="cite_ref-:0bali_1-0"><a href="#cite_note-:0bali-1">[1]</a></sup>
 </th>
 <th>Jumlah penduduk (2017)<sup class="reference" id="cite_ref-:0bali_1-1"><a href="#cite_note-:0bali-1">[1]</a></sup>
 </th>
 <th>Kecamatan
 </th>
 <th>Kelurahan/desa
 </th>
 <th>Logo <div class="center"><div class="floatnone"><a class="image" href="/wiki/Berkas:Coat_of_arms_of_Bali.svg"><img alt="" data-file-height="482" data-file-width="506" decoding="async" height="57" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/10/Coat_of_arms_of_Bali.svg/60px-Coat_of_arms_of_Bali.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/10/Coat_of_arms_of_Bali.svg/90px-Coat_of_arms_of_Bali.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/10/Coat_of_arms_of_Bali.svg/120px-Coat_of_arms_of_Bali.svg.png 2x" width="60"/></a></div></

In [72]:

## create df
## get 'th' data for columns name
df = pd.DataFrame(columns=[header])
df

,No.,Kabupaten kota,Pusat pemerintahan,Bupati wali kota,Luas wilayah (km2)[1],Jumlah penduduk (2017)[1],Kecamatan,Kelurahan desa,Logo,Peta lokasi


In [87]:
# find all data entries in 'td
data1 = []
for row in allRows:
  row = []
  for cell in row:
    data = cell.get_text()
    row.append(row)
  data1.append(row)
data1
# rows = []

# for row in dataRows:
#   row = []
#   for cell in row:
#     data = cell.get_text()
#     row.append(data)
#   # rows.append(row)

# row

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]